<a href="https://colab.research.google.com/github/PoliNemkova/LLM_labeling_skills/blob/main/%E2%AD%90_Experiment_for_LLMs_Labeling_1000_samples_HRV_AAAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Housekeeping

In [ ]:
!pip install ipython-autotime
!pip install openai==0.27.0

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
%load_ext autotime
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
time: 504 ms (started: 2024-12-09 19:39:14 +00:00)


In [ ]:
# API access keys

api_keys = pd.read_csv('/content/gdrive/MyDrive/Computing/api_keys.csv')
api_keys = pd.DataFrame(api_keys)
openai_api_key = api_keys['key'][api_keys['api'] =='openai'].iloc[0]
claude_api_key = api_keys['key'][api_keys['api'] =='claude'].iloc[0]

time: 6.48 ms (started: 2024-12-09 23:43:12 +00:00)


##HRV TESTS

### Data

In [ ]:
# loading annotated dataset

import pandas as pd
human_annotations = pd.read_csv('/content/gdrive/MyDrive/HRV NLP Project X/Data/Data [January 2024]/Annotations/all_adjudicated.csv')

# declare how many samples to use
n = 1000
#n = len(human_annotations)
human_annotations = human_annotations[:n]
human_labels = human_annotations['Adjudicated HRV label'].tolist()
data = human_annotations['Post'].tolist()

time: 38.5 ms (started: 2024-12-09 23:38:19 +00:00)


In [ ]:
len(human_annotations)

1000

time: 2.77 ms (started: 2024-12-09 18:51:22 +00:00)


In [ ]:
# get language breakdow in the sample set


!pip install langdetect

from langdetect import detect, DetectorFactory
import pandas as pd

# Ensure consistent results by setting a seed
DetectorFactory.seed = 0

data = data
df = pd.DataFrame(data)

# Detect language for each string
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

human_annotations['language'] = human_annotations['Post'].apply(detect_language)

# Get breakdown
language_counts = human_annotations['language'].value_counts()

# Display results
print(language_counts)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=065ca705c424c0cd4c4c0d990c6b58bc3f1846fd1902cd1dcbd4bfae60ec17d7
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
language
ru    966
uk     31
bg      2
en      1
Name: count, dtype: int64
time: 9.46 s (started: 2024-12-10 01:40:00 +00:00)


In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(human_annotations['HRV 1'].to_list(), human_annotations['HRV 2'].to_list())
print(f"Cohen's Kappa: {kappa}")

Cohen's Kappa: 0.6309607855721713
time: 1.85 s (started: 2024-12-09 18:51:22 +00:00)


In [ ]:
human_agreement = human_annotations[human_annotations['HRV 1']==human_annotations['HRV 2']]
human_disagreement = human_annotations[human_annotations['HRV 1']!=human_annotations['HRV 2']]
human_annotations = pd.concat([human_agreement, human_disagreement], ignore_index=True) # I want to have forst 85 row with perfect agreement first, and then 15 with disagreement
len(human_disagreement)

184

time: 6 ms (started: 2024-12-09 18:51:24 +00:00)


In [ ]:
human_annotations['Adjudicated HRV label'].value_counts()

,count
Adjudicated HRV label,
Yes,517
No,483


time: 6.86 ms (started: 2024-12-09 18:51:24 +00:00)


###GPT 3.5

####GPT 3.5 Few Shot, eng prompt

In [ ]:
# GPT 3.5 Few Shot Learning -- ENG instructions, WITH EXAMPLES

# had $12.74 on openAI account before running this, ran 5 mins, ended up with $12.48 (cost is $0.26?)
import openai

name = 'GPT_3.5_Few_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.


Here are some examples:

Example 1

Post: In Mykolaiv, after the arrival at the house, there are already five dead…
Label: Yes

Example 2

Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
Label: Yes

Example 3

Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
Label: No

Note that the destruction of military objects (for example, an airbase) is not considered HRV.

You need to output label only."""




# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)


labels_eng_prompt = pd.DataFrame(labels_eng_prompt)



# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()



annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


y_true = human_labels
y_pred = normalize_label(gpt_labels)


print("Model: ", name)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Model:  GPT_3.5_Few_Shot_eng_prompt
Accuracy: 0.661
Precision: 0.6101485148514851
Recall: 0.9535783365570599
F1 Score: 0.7441509433962264
time: 325 ms (started: 2024-12-08 18:23:58 +00:00)


####GPT 3.5 Few Shot, rus prompt

In [ ]:
# GPT 3.5 Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $12.48, ended with $12.05 ($0.43)
import openai

name = 'GPT_3.5_Few_Shot_rus_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT
prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека. Получив публикацию в социальных сетях, ваша задача — классифицировать,
содержит ли она упоминания о нарушениях прав человека. Нарушения включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов, 3) Изнасилование, пытки, казни.
Вы должны возвращать только лейбл "Да" или "Нет".

Примеры:

Пример 1

Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
лейбл: Да

Пример 2

Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
лейбл: Да

Пример 3

Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
лейбл: Нет

Учтите, что уничтожение военных объектов (например, авиабазы) не считается нарушением прав человека.

Вы должны предоставить только лейбл."""


#data = data['Post'].to_list()

# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)


labels_eng_prompt = pd.DataFrame(labels_eng_prompt)


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Accuracy: 0.703
Precision: 0.6498637602179836
Recall: 0.9226305609284333
F1 Score: 0.762589928057554
Model:  GPT_3.5_Few_Shot_rus_prompt
Accuracy: 0.703
Precision: 0.6498637602179836
Recall: 0.9226305609284333
F1 Score: 0.762589928057554
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 6min 1s (started: 2024-12-08 18:27:15 +00:00)


####GPT 3.5 One Shot, eng prompt

In [ ]:
# GPT 3.5 One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $12.05, ended with $11.90 ($0.15)
import openai

name = 'GPT_3.5_One_Shot_eng_prompt'
# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label. """



# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_3.5_One_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.726
Precision: 0.678939617083947
Recall: 0.8916827852998066
F1 Score: 0.7709030100334449
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 5min 9s (started: 2024-12-08 18:40:05 +00:00)


#### GPT 3.5 One Shot, rus prompt

In [ ]:
# GPT 3.5 One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $11.90, ended with $11.76, 5min44sec
import openai

name = 'GPT_3_5_One_Shot_rus_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT
prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека.
Получив публикацию в социальных сетях, ваша задача — классифицировать, содержит ли она упоминания о нарушениях прав человека.
Нарушения прав человека включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов,
3) Изнасилование, пытки, казни. Вы должны возвращать только лейбл "Да" или "Нет" """



# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)
labels_eng_prompt = pd.DataFrame(labels_eng_prompt)


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_3_5_One_Shot_rus_prompt
Sample size:  1000
Accuracy: 0.723
Precision: 0.6851851851851852
Recall: 0.8588007736943907
F1 Score: 0.7622317596566524
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 5min 44s (started: 2024-12-08 18:49:10 +00:00)


### GPT 4o

####GPT 4o Few Shot, eng prompt

In [ ]:
#  GPT 4o Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $11.76, ended with $10.58, took 7min 16s

import openai

name = 'GPT_4o_Few_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.
Here are some examples:

Example 1

Post: In Mykolaiv, after the arrival at the house, there are already five dead…
Label: Yes

Example 2

Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
Label: Yes

Example 3

Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
Label: No

Note that the destruction of military objects (for example, an airbase) is not considered HRV.

You need to putput label only."""


#data = data['Post'].to_list()

# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_Few_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.823
Precision: 0.8556485355648535
Recall: 0.7911025145067698
F1 Score: 0.8221105527638191
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 7min 16s (started: 2024-12-08 18:57:39 +00:00)


####GPT 4o Few Shot, rus prompt

In [ ]:
#  GPT 4o Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $10.58, ended with $9.28, took 5min 40s

import openai

name = 'GPT_4o_Few_Shot_rus_prompt'
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT

prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека. Получив публикацию в социальных сетях, ваша задача — классифицировать,
содержит ли она упоминания о нарушениях прав человека. Нарушения включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов, 3) Изнасилование, пытки, казни.
Вы должны возвращать только лейбл "Да" или "Нет".

Примеры:

Пример 1

Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
лейбл: Да

Пример 2

Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
лейбл: Да

Пример 3

Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
лейбл: Нет

Учтите, что уничтожение военных объектов (например, авиабазы) не считается нарушением прав человека.

Вы должны предоставить только лейбл."""



# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_Few_Shot_rus_prompt
Sample size:  1000
Accuracy: 0.828
Precision: 0.8709677419354839
Recall: 0.7833655705996132
F1 Score: 0.824847250509165
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 5min 40s (started: 2024-12-08 19:06:14 +00:00)


####GPT 4o One Shot, eng prompt

In [ ]:
#  GPT 4o One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $9.28, ended with $8.72
import openai

name = 'GPT_4o_One_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label. """


# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Model:  GPT_4o_One_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.823
Precision: 0.7777777777777778
Recall: 0.9206963249516441
F1 Score: 0.8432240921169176
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 7min 7s (started: 2024-12-08 19:12:59 +00:00)


####GPT 4o One Shot, rus prompt

In [ ]:
#  GPT 4o One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $8.72, ended with $8.10, took 8min 27s
import openai

name = 'GPT_4o_One_Shot_rus_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT
prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека.
Получив публикацию в социальных сетях, ваша задача — классифицировать, содержит ли она упоминания о нарушениях прав человека.
Нарушения прав человека включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов,
3) Изнасилование, пытки, казни. Вы должны возвращать только лейбл "Да" или "Нет" """


# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_One_Shot_rus_prompt
Sample size:  1000
Accuracy: 0.807
Precision: 0.7709030100334449
Recall: 0.8916827852998066
F1 Score: 0.8269058295964126
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 8min 27s (started: 2024-12-08 19:22:13 +00:00)


### LLAMA 3

####Accessing HuggingFace Gated Repos (Needed for LLAMA and Mistral)

In [ ]:
!git config --global credential.helper store

time: 104 ms (started: 2024-12-08 19:32:40 +00:00)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `LLM annotations study` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved t

#### Setting up 8-bit quantization (needed for LLAMA and Mistral)

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.6 MB/s eta 0:00:00
time: 5.03 s (started: 2024-12-08 19:33:17 +00:00)


In [ ]:
#!pip uninstall bitsandbytes transformers
#!pip install transformers bitsandbytes

####Llama 3 One shot, eng prompt

In [ ]:
# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'LLama3_One_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id

# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system.
  Given a social media post, your job is to classify if the post contains HRV mentions. HRV includes (but not limited to):
  1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions.
  Please classify this post: {text}. You need to return just either "Yes" or "No" label. """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.484
Precision: 1.0
Recall: 0.0019342359767891683
F1 Score: 0.003861003861003861
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 47.8 ms (started: 2024-12-08 19:38:48 +00:00)


In [ ]:
### CHANGING PROMPT v2 ###

# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'LLama3_One_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9  # Ensure deterministic output
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Reading this social media post: "{text}", is is clear that it describes human rights violation case. Yes or no?
  Label (only "Yes" or "No") """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Loading model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:

Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.486
Precision: 0.5157894736842106
Recall: 0.09477756286266925
F1 Score: 0.16013071895424835
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 3min 32s (started: 2024-12-08 19:40:34 +00:00)


In [ ]:
### ANOTHER DIFFERENT PROMPT v3 ###


# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'LLama3_One_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9  # Ensure deterministic output
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  You are an expert in identifying human rights violations in social media posts. Your task is
  to analyze the provided post and determine whether it refers to a human rights violation.

  Read the following post carefully:
  "{text}"

  Respond with a single word: "Yes" if the post describes a human rights violation, or "No" if it
  does not. Avoid providing explanations or additional comments.

  Label (only "Yes" or "No"): """



  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Loading model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:

Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.48
Precision: 0.42857142857142855
Recall: 0.017408123791102514
F1 Score: 0.03345724907063197
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 3min 35s (started: 2024-12-08 19:49:21 +00:00)


In [ ]:
### BEST PERFORMING PROMPT ###

### ANOTHER DIFFERENT PROMPT - v4 ###


# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'LLama3_One_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Does this social media post describe a human rights violation?
  Post: "{text}".
  Respond with "Yes" or "No" only.
  Label: """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Loading model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:

Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.516
Precision: 0.5169230769230769
Recall: 0.9748549323017408
F1 Score: 0.675603217158177
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 3min 37s (started: 2024-12-08 19:56:39 +00:00)


####Llama 3 One shot, rus prompt

In [ ]:
### BEST PERFORMING PROMPT ###

# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'Llama3_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Ваша задача — определить, содержит ли пост из соцсети упоминания о нарушениях прав человека.

  Пост: "{text}"
  Предоставьте лейбл (только "Yes" или "No")
  """

  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_One_shot_rus_prompt
Accuracy: 0.499
Precision: 0.55
Recall: 0.1702127659574468
F1 Score: 0.25997045790251105
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 1min 55s (started: 2024-12-08 20:39:34 +00:00)


In [ ]:
### DIFFERENT RPOMT, v2 ###

# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'Llama3_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Вы — система для обнаружения нарушений прав человека. Нарушения включают:
  1) Убийство мирных жителей, 2) Уничтожение гражданских объектов, 3) Изнасилование, пытки, казни.
  Классифицируйте пост на наличие нарушений прав человека: "{text}". Верните только "Yes" или "No".
  """

  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_One_shot_rus_prompt
Accuracy: 0.483
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 1min 39s (started: 2024-12-08 20:47:20 +00:00)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


####Llama 3 Few shot, eng prompt

In [ ]:
# Llama 3 Few shot learning -- ENG prompt, WITH EXAMPLES

name = 'Llama3_Few_shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):
  prompt = f"""Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
  if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.
  Here are some examples:

  Example 1
  Post: In Mykolaiv, after the arrival at the house, there are already five dead…
  Label: Yes

  Example 2
  Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
  Label: Yes

  Example 3
  Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
  Label: No

  Note that the destruction of military objects (for example, an airbase) is not considered HRV.
  Post: {text}
  Label (only "Yes" or "No"):
  """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")


print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")



Model:  Llama3_Few_shot_eng_prompt
Accuracy: 0.526
Precision: 0.52218782249742
Recall: 0.9787234042553191
F1 Score: 0.6810228802153432
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 44 ms (started: 2024-12-08 21:15:34 +00:00)


#### Llama 3 Few shot, rus prompt

In [ ]:
# Llama 3 Few shot learning -- RUS prompt, WITH EXAMPLES

name = 'Llama3_Few_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Инструкция: Вы — гуманитарная система для обнаружения нарушений прав человека. Ваша задача — классифицировать, содержит ли публикация в социальных сетях упоминания о нарушениях прав человека.

  Что считается нарушением прав человека:
  - Убийство мирных жителей
  - Уничтожение гражданских объектов
  - Изнасилование, пытки, казни

  Пример 1:
  Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
  Лейбл: Да

  Пример 2:
  Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
  Лейбл: Да

  Пример 3:
  Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
  Лейбл: Нет

  Теперь классифицируйте следующий пост:
  Пост: "{text}"

  Ответьте только "Да" или "Нет":
  """



  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_Few_shot_rus_prompt
Accuracy: 0.515
Precision: 0.5160642570281124
Recall: 0.9941972920696325
F1 Score: 0.6794448116325181
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 3min 42s (started: 2024-12-08 21:27:52 +00:00)


### MISTRAL-7B

####Mistral-7B One shot, eng prompt

In [ ]:
# MISTRAL One shot learning -- ENG prompt, NO EXAMPLES

name = 'Mistral7B_One_shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data  # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id

# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):
  prompt = f"""Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system.
  Given a social media post, your job is to classify if the post contains HRV mentions. HRV includes (but not limited to):
  1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions.
  You need to return just either "Yes" or "No" label: {text}"""

  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Loading Mistral model with 8-bit quantization...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open

Model:  Mistral7B_One_shot_eng_prompt
Accuracy: 0.484
Precision: 0.6666666666666666
Recall: 0.0038684719535783366
F1 Score: 0.007692307692307693
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 11min 26s (started: 2024-12-08 21:35:51 +00:00)


In [ ]:
### DIFFERENT PROMPT v2

# MISTRAL One shot learning -- ENG prompt, NO EXAMPLES

name = 'Mistral7B_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data  # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):
  prompt = f"""
  Ваша задача — определить, содержит ли пост из соцсети упоминания о нарушениях прав человека.

  Пост: "{text}"
  Предоставьте лейбл (только "Yes" или "No")
  """

  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading Mistral model with 8-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model:  Mistral7B_One_shot_rus_prompt
Accuracy: 0.496
Precision: 0.6756756756756757
Recall: 0.048355899419729204
F1 Score: 0.09025270758122744
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 10min 8s (started: 2024-12-08 22:15:33 +00:00)


In [ ]:
### DIFFERENT PROMPT v3 ###

# MISTRAL One shot learning -- ENG prompt, NO EXAMPLES

name = 'Mistral7B_One_shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data  # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):
  prompt = f"""
  Reading this social media post: "{text}", is is clear that it describes human rights violation case. Yes or No?
  Label (only "Yes" or "No")
  """

  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Loading Mistral model with 8-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model:  Mistral7B_One_shot_eng_prompt
Accuracy: 0.477
Precision: 0.4642857142857143
Recall: 0.07543520309477757
F1 Score: 0.129783693843594
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 10min 6s (started: 2024-12-08 22:31:11 +00:00)


#### Mistral-7B One shot, rus prompt

In [ ]:
# MISTRAL One shot learning -- ENG prompt, NO EXAMPLES

name = 'Mistral7B_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data  # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):
  prompt = f"""Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека.
  Получив публикацию в социальных сетях, ваша задача — классифицировать, содержит ли она упоминания о нарушениях прав человека.
  Нарушения прав человека включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов,
  3) Изнасилование, пытки, казни. Вы должны возвращать только лейбл "Да" или "Нет": {text}"""

  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading Mistral model with 8-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model:  Mistral7B_One_shot_rus_prompt
Accuracy: 0.484
Precision: 0.5238095238095238
Recall: 0.02127659574468085
F1 Score: 0.040892193308550186
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 10min 15s (started: 2024-12-08 22:45:05 +00:00)


In [ ]:
### ANOTHER PROMPT VERSION - v2 ###

# MISTRAL One shot learning -- RUS prompt, NO EXAMPLES

name = 'Mistral7B_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data  # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):
  prompt = f"""
  Читая этот пост в социальных сетях: "{text}", можно заметить, что он описывает случай нарушения прав человека. Да или нет?
  Классифицируй пост лейблом (только "Да" или "Нет")
  """
  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading Mistral model with 8-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model:  Mistral7B_One_shot_rus_prompt
Accuracy: 0.51
Precision: 0.516245487364621
Recall: 0.8297872340425532
F1 Score: 0.6364985163204748
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 10min 10s (started: 2024-12-08 22:55:21 +00:00)


#### Mistral-7B Few Shot, eng prompt

In [ ]:
# MISTRAL Few shot learning -- ENG prompt, WITH EXAMPLES

name = 'Mistral7B_Few_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id



mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=10,  # Increase max tokens for better output
    num_return_sequences=1,
    return_full_text=False,
    truncation=True,  # Ensure long prompts are truncated
    do_sample=False,
    temperature=0.7,  # Lower temperature for more focused results
    pad_token_id=tokenizer.eos_token_id
)


# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):

  prompt = f"""Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
  if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions.
  You need to return just either "Yes" or "No" label.
  Here are some examples:

  Example 1
  Post: In Mykolaiv, after the arrival at the house, there are already five dead…
  Label: Yes

  Example 2
  Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
  Label: Yes

  Example 3
  Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
  Label: No

  Note that the destruction of military objects (for example, an airbase) is not considered HRV.

  Post to label (Yes or No): "{text}"
  """


  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #print(label)

  if any(keyword in label for keyword in ['yes', 'да', 'hrv']):
      return 'Yes'
  elif 'no' in label:
      return 'No'
  else:
      return 'No'  # Default to "No" for unexpected outputs


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+name+'1000samples.csv')


# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")



Loading Mistral model with 8-bit quantization...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-67573fd2-56c222fd502deb8c0b125ebb;b3a0ecca-993c-4c6a-b325-35dd912b3b61)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

time: 12.7 s (started: 2024-12-09 19:06:45 +00:00)


#### Mistral-7B Few Shot, rus prompt

In [ ]:
# MISTRAL Few shot learning -- ENG prompt, WITH EXAMPLES


name = 'Mistral7B_Few_Shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset
dataset = data  # Replace with your actual dataset
model_name = "mistralai/Mistral-7B-v0.1"  # Update to the Mistral model

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with 8-bit quantization

print("Loading Mistral model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

# If you want to use the model without 8-bit quantization, uncomment the code below:
'''
print("Loading Mistral model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
mistral_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using Mistral
def one_shot_annotate(text):

  prompt = f"""Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека. Получив публикацию в социальных сетях, ваша задача — классифицировать,
  содержит ли она упоминания о нарушениях прав человека. Нарушения включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов, 3) Изнасилование, пытки, казни.
  Вы должны возвращать только лейбл "Да" или "Нет".

  Пример 1
  Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
  лейбл: Да

  Пример 2
  Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
  лейбл: Да

  Пример 3
  Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
  лейбл: Нет

  Учтите, что уничтожение военных объектов (например, авиабазы) не считается нарушением прав человека.

  Пост: "{text}"
  Вы должны предоставить только лейбл (только "Да" или "Нет"):
  """


  # Generate label using Mistral
  response = mistral_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using Mistral
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+name+'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
mistral_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, mistral_labels)
precision = precision_score(human_labels, mistral_labels, pos_label="Yes")
recall = recall_score(human_labels, mistral_labels, pos_label="Yes")
f1 = f1_score(human_labels, mistral_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading Mistral model with 8-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model:  Mistral7B_Few_Shot_rus_prompt
Accuracy: 0.519
Precision: 0.5204545454545455
Recall: 0.8858800773694391
F1 Score: 0.6556907659269864
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 10min 54s (started: 2024-12-09 01:06:31 +00:00)


### CLAUDE-2

#### Claude-2 One Shot, eng prompt

In [ ]:
# CLAUDE One Shot Learning -- ENG prompt, NO EXAMPLES

# started with $11.07, ended with $8.97 ($2.1 cost!), took 19min 5s
!pip install anthropic

name = 'Claude2_One_Shot_eng_prompt'

import anthropic
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset

# Initialize the Claude API client
API_KEY = claude_api_key
client = anthropic.Client(api_key=API_KEY)

# Define a function for one-shot annotation using Claude
def one_shot_annotate_claude(text):
    prompt = f"""
    You are a detection system for identifying Human Rights Violations (HRV) in text.
    Reading the following social media post, decide whether there is a reference to a human rights violation:

    Post: "{text}"

    If the post contains any mention of HRV, respond with "Yes". Otherwise, respond with "No":
    """

    # Query Claude
    response = client.completions.create(
        model="claude-2",
        prompt=f"{anthropic.HUMAN_PROMPT} {prompt}{anthropic.AI_PROMPT}",
        max_tokens_to_sample=10,
        temperature=0.1  # Low temperature for deterministic output
    )

    # Extract and clean up the label
    label = response.completion.strip().lower()
    #print(label)
    #return label

    # Normalize and classify the output
    if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
        return 'Yes'
    else:
        return 'No'


# Annotate the dataset using Claude
annotations = [one_shot_annotate_claude(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
claude_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, claude_labels)
precision = precision_score(human_labels, claude_labels, pos_label="Yes")
recall = recall_score(human_labels, claude_labels, pos_label="Yes")
f1 = f1_score(human_labels, claude_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 6.0 MB/s eta 0:00:00
Model:  Claude2_One_Shot_eng_prompt
Accuracy: 0.673
Precision: 0.786144578313253
Recall: 0.504835589941973
F1 Score: 0.6148409893992933
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 19min 5s (started: 2024-12-09 01:50:25 +00:00)


#### Claude-2 One Shot, rus prompt

In [ ]:
# CLAUDE One Shot Learning -- ENG prompt, NO EXAMPLES

# started with $8.97, ended with $5.14, ($3.83 cost!), 19min 11s
#!pip install anthropic

name = 'Claude2_One_Shot_rus_prompt'

import anthropic
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Load your dataset
#n = 20
dataset = data # Replace with your actual dataset

# Initialize the Claude API client
API_KEY = claude_api_key
client = anthropic.Client(api_key=API_KEY)

# Define a function for one-shot annotation using Claude
def one_shot_annotate_claude(text):
    prompt = fprompt = f"""Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека.
    Получив публикацию в социальных сетях, ваша задача — классифицировать, содержит ли она упоминания о нарушениях прав человека.
    Нарушения прав человека включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов,
    3) Изнасилование, пытки, казни. Вы должны возвращать только лейбл "Да" или "Нет". Пост: {text}"""


    # Query Claude
    response = client.completions.create(
        model="claude-2",
        prompt=f"{anthropic.HUMAN_PROMPT} {prompt}{anthropic.AI_PROMPT}",
        max_tokens_to_sample=10,
        temperature=0.1  # Low temperature for deterministic output
    )

    # Extract and clean up the label
    label = response.completion.strip().lower()
    #return label

    # Normalize and classify the output
    if 'yes' in label or 'да' in label or 'hrv' in label:
        return 'Yes'
    else:
        return 'No'


# Annotate the dataset using Claude
annotations = [one_shot_annotate_claude(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
claude_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, claude_labels)
precision = precision_score(human_labels, claude_labels, pos_label="Yes")
recall = recall_score(human_labels, claude_labels, pos_label="Yes")
f1 = f1_score(human_labels, claude_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Model:  Claude2_One_Shot_rus_prompt
Accuracy: 0.427
Precision: 0.4455252918287938
Recall: 0.44294003868471954
F1 Score: 0.44422890397672166
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt
time: 19min 11s (started: 2024-12-09 02:14:02 +00:00)


#### Claude-2 Few Shot, eng prompt

In [ ]:
# CLAUDE Few Shot Learning -- ENG prompt, WITH EXAMPLES

# started with $12.75,

#!pip install anthropic

name = 'Claude2_Few_Shot_eng_prompt'

import anthropic
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset

# Initialize the Claude API client
API_KEY = claude_api_key
client = anthropic.Client(api_key=API_KEY)

# Define a function for one-shot annotation using Claude
def one_shot_annotate_claude(text):
    prompt = f"""
    You are a detection system for identifying Human Rights Violations (HRV) in text. You will have to annotate posts.
     Here are some examples:

    Example 1

    Post: In Mykolaiv, after the arrival at the house, there are already five dead…
    Label: Yes

    Example 2

    Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
    Label: Yes

    Example 3

    Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
    Label: No

    Note that the destruction of military objects (for example, an airbase) is not considered HRV.

    Reading the following social media post, decide whether it clearly describes a human rights violation:

    Post: "{text}"

    If the post contains any mention of HRV, respond with "Yes". Otherwise, respond with "No".

    Label (only "Yes" or "No"):
    """

   # Query Claude
    response = client.completions.create(
        model="claude-2",
        prompt=f"{anthropic.HUMAN_PROMPT} {prompt}{anthropic.AI_PROMPT}",
        max_tokens_to_sample=10,
        temperature=0.1  # Low temperature for deterministic output
    )

    """

    response = anthropic.Anthropic().messages.create(
        model="claude-3-5-sonnet-20241022",
        messages=f" {prompt}",
        max_tokens=10,
        temperature=0.1  # Low temperature for deterministic output
    )
    """
    # Extract and clean up the label
    label = response.completion.strip().lower()
    #return label

    # Normalize and classify the output
    if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
        return 'Yes'
    else:
        return 'No'


# Annotate the dataset using Claude
annotations = [one_shot_annotate_claude(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
claude_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, claude_labels)
precision = precision_score(human_labels, claude_labels, pos_label="Yes")
recall = recall_score(human_labels, claude_labels, pos_label="Yes")
f1 = f1_score(human_labels, claude_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


InternalServerError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

time: 5min 42s (started: 2024-12-09 23:44:22 +00:00)


#### Claude-2 Few Shot, rus prompt

In [ ]:
# CLAUDE Few Shot Learning -- ENG prompt, WITH EXAMPLES

#!pip install anthropic

name = 'Claude2_Few_Shot_rus_prompt'

import anthropic
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Set a random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset

# Initialize the Claude API client
API_KEY = claude_api_key
client = anthropic.Client(api_key=API_KEY)

# Define a function for one-shot annotation using Claude
def one_shot_annotate_claude(text):
    prompt = f"""
    Ваша задача классифировать пост из соцсети на наличине нарушений прав человека.
    Примеры:

    Пример 1

    Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
    лейбл: Да

    Пример 2

    Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
    лейбл: Да

    Пример 3

    Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
    лейбл: Нет

    Учтите, что уничтожение военных объектов (например, авиабазы) не считается нарушением прав человека.

    Вы должны предоставить только лейбл.

    Читая этот пост в социальных сетях: "{text}", можно заметить, что он описывает случай нарушения прав человека. Верно?
    Классифицируй пост лейблом (только "Да" или "Нет")
    """


    # Query Claude
    response = client.completions.create(
        model="claude-2",
        prompt=f"{anthropic.HUMAN_PROMPT} {prompt}{anthropic.AI_PROMPT}",
        max_tokens_to_sample=10,
        temperature=0.1  # Low temperature for deterministic output
    )

    # Extract and clean up the label
    label = response.completion.strip().lower()
    #return label

    # Normalize and classify the output
    if 'yes' in label or 'да' in label or 'hrv' in label:
        return 'Yes'
    else:
        return 'No'


# Annotate the dataset using Claude
annotations = [one_shot_annotate_claude(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/drive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
claude_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, claude_labels)
precision = precision_score(human_labels, claude_labels, pos_label="Yes")
recall = recall_score(human_labels, claude_labels, pos_label="Yes")
f1 = f1_score(human_labels, claude_labels, pos_label="Yes")

# Display the evaluation metrics
print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


InternalServerError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

time: 2.61 s (started: 2024-12-10 00:11:32 +00:00)


## **ABLATION STUDY**

##### DATA

In [ ]:
# data

human_agreement = human_annotations[human_annotations['HRV 1']==human_annotations['HRV 2']]
human_disagreement = human_annotations[human_annotations['HRV 1']!=human_annotations['HRV 2']]
human_annotations = pd.concat([human_agreement, human_disagreement], ignore_index=True) # I want to have forst 85 row with perfect agreement first, and then 15 with disagreement
len(human_disagreement)

184

time: 5.96 ms (started: 2024-12-09 19:39:26 +00:00)


In [ ]:
# labels

data_agreement = human_agreement['Post'].to_list()
human_agreement_annotations = human_agreement['Adjudicated HRV label'].to_list()

data_disagreement = human_disagreement['Post'].tolist()
human_disagreement_annotations = human_disagreement['Adjudicated HRV label'].to_list()

time: 873 µs (started: 2024-12-09 19:39:26 +00:00)


In [ ]:
# setting up new results doc to write to

results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt"

time: 389 µs (started: 2024-12-09 19:39:26 +00:00)


##### Agreement experiment first

In [ ]:
data = data_agreement
human_labels = human_agreement_annotations

time: 423 µs (started: 2024-12-09 22:20:37 +00:00)


##### LLAMA 3

###### Accessing HuggingFace Gated Repos (Needed for LLAMA and Mistral)

In [ ]:
!git config --global credential.helper store

time: 104 ms (started: 2024-12-09 19:39:38 +00:00)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

###### Setting up 8-bit quantization (needed for LLAMA and Mistral)

In [ ]:
!pip install -U bitsandbytes

time: 2.51 s (started: 2024-12-09 19:37:31 +00:00)


In [ ]:
!pip uninstall bitsandbytes transformers
!pip install transformers bitsandbytes

##### Llama 3 One shot, eng prompt

In [ ]:
# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'LLama3_One_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Does this social media post describe a human rights violation?
  Post: "{text}".
  Respond with "Yes" or "No" only.
  Label: """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"
# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.5318627450980392
Precision: 0.533083645443196
Recall: 0.9816091954022989
F1 Score: 0.6909385113268608
time: 1.06 ms (started: 2024-12-09 19:47:04 +00:00)


##### Llama 3 One shot, rus prompt

In [ ]:
# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'Llama3_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Ваша задача — определить, содержит ли пост из соцсети упоминания о нарушениях прав человека.

  Пост: "{text}"
  Предоставьте лейбл (только "Yes" или "No")
  """

  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples_agreement.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_One_shot_rus_prompt
Accuracy: 0.48161764705882354
Precision: 0.5454545454545454
Recall: 0.16551724137931034
F1 Score: 0.25396825396825395
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 1min 34s (started: 2024-12-09 22:21:10 +00:00)


####Llama 3 Few shot, eng prompt

In [ ]:
# Llama 3 Few shot learning -- ENG prompt, WITH EXAMPLES

name = 'Llama3_Few_shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):
  prompt = f"""Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
  if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.
  Here are some examples:

  Example 1
  Post: In Mykolaiv, after the arrival at the house, there are already five dead…
  Label: Yes

  Example 2
  Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
  Label: Yes

  Example 3
  Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
  Label: No

  Note that the destruction of military objects (for example, an airbase) is not considered HRV.
  Post: {text}
  Label (only "Yes" or "No"):
  """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")


print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")



Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_Few_shot_eng_prompt
Accuracy: 0.5428921568627451
Precision: 0.5389447236180904
Recall: 0.9862068965517241
F1 Score: 0.6969943135662063
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 2min 58s (started: 2024-12-09 22:27:56 +00:00)


#### Llama 3 Few shot, rus prompt

In [ ]:
# Llama 3 Few shot learning -- RUS prompt, WITH EXAMPLES

name = 'Llama3_Few_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Инструкция: Вы — гуманитарная система для обнаружения нарушений прав человека. Ваша задача — классифицировать, содержит ли публикация в социальных сетях упоминания о нарушениях прав человека.

  Что считается нарушением прав человека:
  - Убийство мирных жителей
  - Уничтожение гражданских объектов
  - Изнасилование, пытки, казни

  Пример 1:
  Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
  Лейбл: Да

  Пример 2:
  Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
  Лейбл: Да

  Пример 3:
  Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
  Лейбл: Нет

  Теперь классифицируйте следующий пост:
  Пост: "{text}"

  Ответьте только "Да" или "Нет":
  """



  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_Few_shot_rus_prompt
Accuracy: 0.5318627450980392
Precision: 0.5325153374233129
Recall: 0.9977011494252873
F1 Score: 0.6944
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 3min 1s (started: 2024-12-09 22:31:06 +00:00)


#### GPT4o

####GPT 4o Few Shot, eng prompt

In [ ]:
#  GPT 4o Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $11.76, ended with $10.58, took 7min 16s

import openai

name = 'GPT_4o_Few_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.
Here are some examples:

Example 1

Post: In Mykolaiv, after the arrival at the house, there are already five dead…
Label: Yes

Example 2

Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
Label: Yes

Example 3

Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
Label: No

Note that the destruction of military objects (for example, an airbase) is not considered HRV.

You need to putput label only."""


#data = data['Post'].to_list()

# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_Few_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.8602941176470589
Precision: 0.8962962962962963
Recall: 0.8344827586206897
F1 Score: 0.8642857142857143
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 6min 9s (started: 2024-12-09 22:36:35 +00:00)


####GPT 4o Few Shot, rus prompt

In [ ]:
#  GPT 4o Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $10.58, ended with $9.28, took 5min 40s

import openai

name = 'GPT_4o_Few_Shot_rus_prompt'
# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT

prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека. Получив публикацию в социальных сетях, ваша задача — классифицировать,
содержит ли она упоминания о нарушениях прав человека. Нарушения включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов, 3) Изнасилование, пытки, казни.
Вы должны возвращать только лейбл "Да" или "Нет".

Примеры:

Пример 1

Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
лейбл: Да

Пример 2

Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
лейбл: Да

Пример 3

Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
лейбл: Нет

Учтите, что уничтожение военных объектов (например, авиабазы) не считается нарушением прав человека.

Вы должны предоставить только лейбл."""



# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", len(data))
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {len(data)}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_Few_Shot_rus_prompt
Sample size:  816
Accuracy: 0.8578431372549019
Precision: 0.9164490861618799
Recall: 0.8068965517241379
F1 Score: 0.8581907090464548
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 5min 34s (started: 2024-12-09 22:44:45 +00:00)


####GPT 4o One Shot, eng prompt

In [ ]:
#  GPT 4o One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $9.28, ended with $8.72
import openai

name = 'GPT_4o_One_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label. """


# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Model:  GPT_4o_One_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.8468137254901961
Precision: 0.8163265306122449
Recall: 0.9195402298850575
F1 Score: 0.8648648648648649
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 6min 28s (started: 2024-12-09 22:58:29 +00:00)


####GPT 4o One Shot, rus prompt

In [ ]:
#  GPT 4o One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $8.72, ended with $8.10, took 8min 27s
import openai

name = 'GPT_4o_One_Shot_rus_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT
prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека.
Получив публикацию в социальных сетях, ваша задача — классифицировать, содержит ли она упоминания о нарушениях прав человека.
Нарушения прав человека включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов,
3) Изнасилование, пытки, казни. Вы должны возвращать только лейбл "Да" или "Нет" """


# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_One_Shot_rus_prompt
Sample size:  1000
Accuracy: 0.8419117647058824
Precision: 0.8200836820083682
Recall: 0.9011494252873563
F1 Score: 0.8587075575027382
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 7min 35s (started: 2024-12-09 23:06:24 +00:00)


#### Disagreement test

In [ ]:
data = data_disagreement
human_labels = human_disagreement_annotations

time: 395 µs (started: 2024-12-09 23:17:39 +00:00)


##### LLAMA 3

###### Accessing HuggingFace Gated Repos (Needed for LLAMA and Mistral)

In [ ]:
!git config --global credential.helper store

time: 104 ms (started: 2024-12-09 19:39:38 +00:00)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

###### Setting up 8-bit quantization (needed for LLAMA and Mistral)

In [ ]:
!pip install -U bitsandbytes

time: 2.51 s (started: 2024-12-09 19:37:31 +00:00)


In [ ]:
!pip uninstall bitsandbytes transformers
!pip install transformers bitsandbytes

##### Llama 3 One shot, eng prompt

In [ ]:
# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'LLama3_One_Shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Does this social media post describe a human rights violation?
  Post: "{text}".
  Respond with "Yes" or "No" only.
  Label: """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.44565217391304346
Precision: 0.4425287356321839
Recall: 0.9390243902439024
F1 Score: 0.6015625
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
Model:  LLama3_One_Shot_eng_prompt
Accuracy: 0.44565217391304346
Precision: 0.4425287356321839
Recall: 0.9390243902439024
F1 Score: 0.6015625
time: 41.5 s (started: 2024-12-09 23:17:52 +00:00)


##### Llama 3 One shot, rus prompt

In [ ]:
# Llama 3 One shot learning -- ENG prompt, NO EXAMPLES

name = 'Llama3_One_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Ваша задача — определить, содержит ли пост из соцсети упоминания о нарушениях прав человека.

  Пост: "{text}"
  Предоставьте лейбл (только "Yes" или "No")
  """

  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})

annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples_agreement.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_One_shot_rus_prompt
Accuracy: 0.5760869565217391
Precision: 0.5714285714285714
Recall: 0.1951219512195122
F1 Score: 0.2909090909090909
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 23.4 s (started: 2024-12-09 23:18:43 +00:00)


####Llama 3 Few shot, eng prompt

In [ ]:
# Llama 3 Few shot learning -- ENG prompt, WITH EXAMPLES

name = 'Llama3_Few_shot_eng_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):
  prompt = f"""Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
  if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.
  Here are some examples:

  Example 1
  Post: In Mykolaiv, after the arrival at the house, there are already five dead…
  Label: Yes

  Example 2
  Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
  Label: Yes

  Example 3
  Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
  Label: No

  Note that the destruction of military objects (for example, an airbase) is not considered HRV.
  Post: {text}
  Label (only "Yes" or "No"):
  """


  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")


print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")



Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_Few_shot_eng_prompt
Accuracy: 0.45108695652173914
Precision: 0.44508670520231214
Recall: 0.9390243902439024
F1 Score: 0.6039215686274509
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 43.3 s (started: 2024-12-09 23:20:03 +00:00)


#### Llama 3 Few shot, rus prompt

In [ ]:
# Llama 3 Few shot learning -- RUS prompt, WITH EXAMPLES

name = 'Llama3_Few_shot_rus_prompt'

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from datasets import load_dataset
import torch
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold


# Set a random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load your dataset

dataset = data  # Replace with your actual dataset
#model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Llama-3.2-1B"

# Configure the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with the new quantization config

print("Loading model with 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)
''' # if I want to use the model withought 8-bit quantization
print("Loading model WITHOUT 8-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)
'''

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad_token_id to eos_token_id to avoid warnings
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.eos_token_id


# Create the pipeline with greedy decoding for consistency
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3,
    num_return_sequences=1,
    return_full_text=False,
    do_sample=False,  # Greedy decoding
    temperature=0.9,  # Ensure deterministic output
    pad_token_id=tokenizer.eos_token_id
)

# Define a function for one-shot annotation using LLaMA
def one_shot_annotate(text):

  prompt = f"""
  Инструкция: Вы — гуманитарная система для обнаружения нарушений прав человека. Ваша задача — классифицировать, содержит ли публикация в социальных сетях упоминания о нарушениях прав человека.

  Что считается нарушением прав человека:
  - Убийство мирных жителей
  - Уничтожение гражданских объектов
  - Изнасилование, пытки, казни

  Пример 1:
  Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
  Лейбл: Да

  Пример 2:
  Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
  Лейбл: Да

  Пример 3:
  Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
  Лейбл: Нет

  Теперь классифицируйте следующий пост:
  Пост: "{text}"

  Ответьте только "Да" или "Нет":
  """



  # Generate label using LLaMA
  response = llama_pipeline(prompt, truncation=True)
  label = response[0]['generated_text'].strip().lower()
  #return label

  # Normalize and classify the output
  if 'yes' in label or 'да' in label or 'hrv' in label or 'Yes' in label:
      return 'Yes'
  else:
      return 'No'


# Annotate the dataset using LLaMA
annotations = [one_shot_annotate(text) for text in data]

# Save annotations to a DataFrame
annotations_df = pd.DataFrame({
    'text': data,
    'annotation': annotations,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000samples.csv')

# Evaluation against ground truth
#human_labels = dataset['HRV label'].tolist()
llama_labels = annotations_df['annotation'].tolist()

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(human_labels, llama_labels)
precision = precision_score(human_labels, llama_labels, pos_label="Yes")
recall = recall_score(human_labels, llama_labels, pos_label="Yes")
f1 = f1_score(human_labels, llama_labels, pos_label="Yes")

print("Model: ", name)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Loading model with 8-bit quantization...


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Model:  Llama3_Few_shot_rus_prompt
Accuracy: 0.44565217391304346
Precision: 0.44565217391304346
Recall: 1.0
F1 Score: 0.6165413533834586
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 43.9 s (started: 2024-12-09 23:22:49 +00:00)


#### GPT4o

####GPT 4o Few Shot, eng prompt

In [ ]:
#  GPT 4o Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $11.76, ended with $10.58, took 7min 16s

import openai

name = 'GPT_4o_Few_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label.
Here are some examples:

Example 1

Post: In Mykolaiv, after the arrival at the house, there are already five dead…
Label: Yes

Example 2

Post: Russian aviation struck the Neptune pool in Mariupol where there were pregnant women and mothers with children under 3 years old. Orcs fighting against civilians - a special cauldron in hell is prepared for you. Informer | Subscribe
Label: Yes

Example 3

Post: The first episode is played. There will be others. And more. I will express a personal position. I cannot fail to mention this now. The Ukrainian state in its current configuration with a Nazi political regime will pose a constant, direct, and clear threat to Russia. Therefore, in addition to protecting our people and guarding the country's borders, the goal of our future actions, in my opinion, should be the full dismantling of the political regime in Ukraine.
Label: No

Note that the destruction of military objects (for example, an airbase) is not considered HRV.

You need to putput label only."""


#data = data['Post'].to_list()

# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_Few_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.6793478260869565
Precision: 0.6533333333333333
Recall: 0.5975609756097561
F1 Score: 0.6242038216560509
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 1min 16s (started: 2024-12-09 23:23:55 +00:00)


####GPT 4o Few Shot, rus prompt

In [ ]:
#  GPT 4o Few Shot Learning -- ENG instructions, WITH EXAMPLES

# started with $10.58, ended with $9.28, took 5min 40s

import openai

name = 'GPT_4o_Few_Shot_rus_prompt'
# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT

prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека. Получив публикацию в социальных сетях, ваша задача — классифицировать,
содержит ли она упоминания о нарушениях прав человека. Нарушения включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов, 3) Изнасилование, пытки, казни.
Вы должны возвращать только лейбл "Да" или "Нет".

Примеры:

Пример 1

Пост: В Николаеве, после прилёта в дом, уже пятеро погибших...
лейбл: Да

Пример 2

Пост: Российская авиация нанесла удар по бассейну «Нептун» в Мариуполе, где находились беременные женщины и матери с детьми до 3 лет. Орки, воюющие против мирных жителей, — для вас в аду приготовлен особый котел. Информатор | Подписывайтесь.
лейбл: Да

Пример 3

Пост: Первый эпизод сыгран. Будут другие. И еще. Я выражу личную позицию. Не могу не упомянуть об этом сейчас. Украинское государство в его нынешней конфигурации с нацистским политическим режимом будет представлять постоянную, прямую и явную угрозу для России. Поэтому, помимо защиты нашего народа и охраны границ страны, целью наших будущих действий, по моему мнению, должно стать полное демонтаж политического режима в Украине.
лейбл: Нет

Учтите, что уничтожение военных объектов (например, авиабазы) не считается нарушением прав человека.

Вы должны предоставить только лейбл."""



# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", len(data))
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {len(data)}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_Few_Shot_rus_prompt
Sample size:  184
Accuracy: 0.6739130434782609
Precision: 0.6486486486486487
Recall: 0.5853658536585366
F1 Score: 0.6153846153846154
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 1min 18s (started: 2024-12-09 23:26:02 +00:00)


####GPT 4o One Shot, eng prompt

In [ ]:
#  GPT 4o One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $9.28, ended with $8.72
import openai

name = 'GPT_4o_One_Shot_eng_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL ENGLIGH PROMPT
prompt = """Instruction: You are a humanitarian state-of-the-art Human Rights Violation (HRV) detection system. Given a social media post, your job is to classify
if the post contains HRV mentions. HRV includes (but not limited to): 1) Killing civilians, 2) Destruction of civil objects, 3) Rape, torture, executions. You need to return just either "Yes" or "No" label. """


# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

#labels_eng_prompt.to_csv('/content/gdrive/My Drive/HRV NLP Project X/Data/Data [January 2024]/gpt_labels_eng_prompt_hrv.csv')


# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")


Model:  GPT_4o_One_Shot_eng_prompt
Sample size:  1000
Accuracy: 0.6195652173913043
Precision: 0.55
Recall: 0.8048780487804879
F1 Score: 0.6534653465346535
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 1min 23s (started: 2024-12-09 23:29:21 +00:00)


####GPT 4o One Shot, rus prompt

In [ ]:
#  GPT 4o One Shot Learning -- ENG instructions, NO EXAMPLES

# started with $8.72, ended with $8.10, took 8min 27s
import openai

name = 'GPT_4o_One_Shot_rus_prompt'

# Set up your OpenAI API key
openai.api_key = openai_api_key

def label_with_chatgpt(prompt, data):
    responses = []
    for item in data:
        full_prompt = f"{prompt}\n\nData: {item}"
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=50
        )
        label = response.choices[0].message['content'].strip()
        responses.append(label)
    return responses


### ALL RUSSIAN PROMPT
prompt = """Инструкция: Вы — современная гуманитарная онлайн система для обнаружения нарушений прав человека.
Получив публикацию в социальных сетях, ваша задача — классифицировать, содержит ли она упоминания о нарушениях прав человека.
Нарушения прав человека включают (но не ограничиваются): 1) Убийство мирных жителей, 2) Уничтожение гражданских объектов,
3) Изнасилование, пытки, казни. Вы должны возвращать только лейбл "Да" или "Нет" """


# Get labels
labels_eng_prompt = label_with_chatgpt(prompt, data)

def normalize_label(labels):
  out = []
  for label in labels:
    label = label.lower()  # Normalize to lowercase
    if 'yes' in label or 'да' in label or 'hrv' in label:  # Check for relevant terms
      out.append('Yes')
    else:
      out.append('No')
  return out

# Apply the normalization function to the list
labels_eng_prompt = normalize_label(labels_eng_prompt)

labels_eng_prompt = pd.DataFrame(labels_eng_prompt)

# evaluation

# encoding
#labels_eng_prompt['labels'] = labels_eng_prompt[0].replace({'Yes': 1, 'No': 0})
gpt_labels = labels_eng_prompt[0].to_list()


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = human_labels
y_pred = gpt_labels


# Calculate accuracy
print("Model: ", name)
print("Sample size: ", n)
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, pos_label="Yes")
recall = recall_score(y_true, y_pred, pos_label="Yes")
f1 = f1_score(y_true, y_pred, pos_label="Yes")

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

annotations_df = pd.DataFrame({
    'text': data,
    'annotation': gpt_labels,
    'human label': human_labels
})
annotations_df.to_csv('/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels/'+ name +'1000.csv')



from datetime import datetime
current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

#results_file = "/content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_1000samples.txt"

# Append results to the text file
with open(results_file, "a") as file:
    file.write(f"Date and Time: {current_datetime}\n")
    file.write(f"Model: {name}\n")
    file.write(f"Samples: {n}\n")
    file.write(f"Prompt: {prompt}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Precision: {precision}\n")
    file.write(f"Recall: {recall}\n")
    file.write(f"F1 Score: {f1}\n")
    file.write("-" * 50 + "\n")  # Separator for readability

print(f"Results written to {results_file}")

Model:  GPT_4o_One_Shot_rus_prompt
Sample size:  1000
Accuracy: 0.8419117647058824
Precision: 0.8200836820083682
Recall: 0.9011494252873563
F1 Score: 0.8587075575027382
Results written to /content/gdrive/MyDrive/RSRCH 2025/LLM Annotation Analysis/experiment_results_agreement_set.txt
time: 7min 35s (started: 2024-12-09 23:06:24 +00:00)


### **ERRORS EVALUTION**

In [ ]:
from google.colab import drive
import os
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Path to the folder containing CSV files
folder_path = '/content/drive/MyDrive/RSRCH 2025/LLM Annotation Analysis/Labels'

# Initialize the combined DataFrame as None
combined_df = None

# Iterate over files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file_name)
        # Remove the .csv extension for the column name
        df_name = os.path.splitext(file_name)[0]
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Check if required columns exist in the CSV
        if not all(col in df.columns for col in ['text', 'human label', 'annotation']):
            print(f"Skipping {file_name} as it does not have the required columns.")
            continue

        # Retain only the required columns
        df = df[['text', 'human label', 'annotation']]
        # Rename the 'annotation' column to the file-specific column name
        df = df.rename(columns={'annotation': df_name})

        # Merge with the combined DataFrame
        if combined_df is None:
            combined_df = df  # Initialize with the first file
        else:
            combined_df = pd.merge(
                combined_df,
                df,
                on=['text', 'human label'],
                how='outer'
            )

# Display the combined DataFrame
if combined_df is not None:
    print("Combined DataFrame created successfully!")
    print(combined_df.head())
else:
    print("No valid CSV files found.")

# Save the combined DataFrame to a CSV if needed
combined_df.to_csv('/content/drive/MyDrive/RSRCH 2025/LLM Annotation Analysis/combined_labeld.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Combined DataFrame created successfully!
                                                text human label  \
0  **12 погибших в результате удара по Запорожью*...         Yes   
1  **29 сентября в окрестностях Кременной был лик...         Yes   
2  **БЕЗ РОССИИ - НИКУДА:**\n"__Два прилета было,...         Yes   
3  **В "демократической" Латвии\n\n**__Сидящий в ...         Yes   
4  **В Истре в коттеджном посёлке «Шервуд» мужчин...          No   

  GPT_3.5_One_Shot_eng_prompt Mistral7B_One_shot_rus_prompt  \
0                         Yes                            No   
1                         Yes                           Yes   
2                         Yes                            No   
3                         Yes                           Yes   
4                         Yes                           Yes   

  Claude2_One_Shot_eng_prompt Claude2_One_

In [ ]:
combined_df.head(3)

,text,human label,GPT_3.5_One_Shot_eng_prompt,Mistral7B_One_shot_rus_prompt,Claude2_One_Shot_eng_prompt,Claude2_One_Shot_rus_prompt,Claude2_Few_Shot_rus_prompt,Claude2_Few_Shot_eng_prompt,Mistral7B_Few_Shot_eng_prompt,GPT_3.5_Few_Shot_eng_prompt,...,GPT_4o_Few_Shot_eng_prompt,GPT_4o_Few_Shot_rus_prompt,GPT_4o_One_Shot_eng_prompt,GPT_4o_One_Shot_rus_prompt,LLama3_One_Shot_eng_prompt,Llama3_Few_shot_eng_prompt,Llama3_Few_shot_rus_prompt,Llama3_One_shot_rus_prompt,Mistral7B_Few_Shot_rus_prompt,Mistral7B_One_shot_eng_prompt
0,**12 погибших в результате удара по Запорожью*...,Yes,Yes,No,Yes,Yes,Yes,Yes,No,Yes,...,Yes,Yes,Yes,Yes,No,Yes,Yes,No,Yes,No
1,**29 сентября в окрестностях Кременной был лик...,Yes,Yes,Yes,No,Yes,Yes,Yes,No,Yes,...,Yes,Yes,Yes,Yes,Yes,No,No,No,No,Yes
2,"**БЕЗ РОССИИ - НИКУДА:**\n""__Два прилета было,...",Yes,Yes,No,No,Yes,Yes,No,No,Yes,...,Yes,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes


time: 16 ms (started: 2024-11-24 19:08:45 +00:00)


In [ ]:
combined_df.to_csv('/content/drive/MyDrive/RSRCH 2025/LLM Annotation Analysis/combined_labels.csv', index=False)

time: 21.2 ms (started: 2024-11-24 19:09:18 +00:00)


##**END HERE**